![SemanticKernel](../assets/semantic-kernel.png)

# Microsoft Semantic Kernel Demo

Es una librería completamente de código abierto (_open-source_) arbitrada por Microsoft que, aunque podríamos decir que nació para competir con LangChain, a evolucionado para ocupar su propio nicho funcional. Proporciona facilidades para integrar aplicaciones con LLMs en varios lenguajes de programación, siendo el principal C#, y luego Python, Java y JavaScript.

## Ejemplo del taller

Vamos a realizar un ejemplo de cómo diseñar e implementar un chatbot impulsado por un modelo de lenguaje grande (LLM, _Large Language Model_), en este caso de Azure OpenAI. Este chatbot podrá mantener una conversación y recordar interacciones previas con un modelo de chat.

Hay que tomar en cuenta que este chatbot que construiremos solo utilizará el modelo de lenguaje para conversar. Más adelante en el taller trataremos otros conceptos relacionados tales como:

 - Generación Aumentada por Recuperación: Que permite una experiencia de chatbot sobre una fuente de datos externa, lo cual es conocido habitualmente por sus siglas en ingles de RAG (_Retrieval Augmented Generation_).
 - Agentes: Construir un chatbot que pueda tomar acciones, algunas veces de forma independiente, para satisfacer un requerimiento específico en un área de dominio.

## Dependencias y Referencias

Primero instalaremos las siguientes dependencias:

 - `dotenv.net` → es una biblioteca de .NET que facilita la gestión de variables de entorno en tus proyectos.
 - `Microsoft.SemanticKernel` → contiene las abstracciones base que impulsan el resto del ecosistema de Semantic Kernel. Estas abstracciones están diseñadas para ser modulares y simples, permitiendo que cualquier proveedor implemente la interfaz requerida y se integre fácilmente con el resto del ecosistema. Ofrece variantes por defecto para la gran mayoría de acciones.
 

Con estas dependencias instaladas, importamos los siguienbtes recursos:

- `dotenv.net` → Importa la capacidad de leer las variables de entorno desde un archivo `.env`. La llamada `DotEnv.Load();` lee el archivo `.env` y carga las variables definidas en él como variables de entorno del sistema. Esto es útil para gestionar configuraciones sensibles como claves API y credenciales sin hardcodearlas en el código fuente.
- `Microsoft.SemanticKernel` → Importa las clases y abstracciones base de Semantic Kernel.
- `Microsoft.SemanticKernel.ChatCompletion` → Importa las clases y abstracciones necesarias para la capacidad de chat en modelos de Inteligencia Artificial tales como `gpt-4o`.
- `Microsoft.SemanticKernel.Connectors.OpenAI` → Importa las integraciones para OpenAI a través de su SDK, tanto de la propia empresa detrás de ChatGPT como la versión de Azure OpenAI (o cualquier otra).

Finalmente leemos las variables de entorno que nos proporcionan los siguientes valores:

- `AZURE_OPENAI_API_KEY` → Es la clave secreta de autenticación y autorización para emplear la instancia del servicio de Azure OpenAI que tengamos desplegado en nuestra instancia de la nube de Microsoft.
- `AZURE_OPENAI_ENDPOINT` → Es la dirección (o URL) desde la cual podemos acceder al servicio de Azure OpenAI que tengamos desplegado en nuestra instancia de la nube de Microsoft.
- `AZURE_OPENAI_DEPLOYMENT_NAME` → Es el nombre del despliegue del modelo (LLM) que vamos a utilizar.
- `AZURE_OPENAI_MODEL_NAME` → Es el nombre (o tipo) del modelo que vamos a utilizar. Por ejemplo `gpt-4o`.
- `AZURE_OPENAI_API_VERSION` → Es la versión del API de Azure con la cual estarmos interactuando con el servicio. Por ejemplo `2024-02-15-preview`.

In [ ]:
// Install the DotNetEnv package using the NuGet package manager
#r "nuget: dotenv.net, 3.2.1"
#r "nuget: Microsoft.SemanticKernel, 1.30.0"


In [ ]:

using dotenv.net;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

 // Load the .env file
DotEnv.Load();


In [ ]:

// Access the environment variables
var azureOpenAIEndpoint = Environment.GetEnvironmentVariable("AZURE_OPENAI_ENDPOINT");
var azureOpenAIKey = Environment.GetEnvironmentVariable("AZURE_OPENAI_API_KEY");
var azureOpenAIModel = Environment.GetEnvironmentVariable("AZURE_OPENAI_MODEL_NAME"); 
var azureOpenAIDeployment = Environment.GetEnvironmentVariable("AZURE_OPENAI_DEPLOYMENT_NAME");
var azureOpenAIApiVersion = Environment.GetEnvironmentVariable("AZURE_OPENAI_API_VERSION");

Una vez que tenemos las dependencias y configuraciones cargadas, configuramos un `Kernel`, el cual representa el componente central de esta tecnología. En su forma más simple, el kernel es un contenedor de inyección de dependencias para Inteligencia Artificial que gestiona todos los servicios y _plugins_ necesarios para ejecutar una aplicación o servicio enriquecido con este tipo de tecnologías. Al proporcionar todos los servicios y _plugins_ al kernel, estos serán utilizados de manera fluida por la IA según sea necesario.

El kernel permite configurar y monitorear aplicaciones de IA desde un solo lugar, lo que facilita la integración y el uso de diferentes servicios y _plugins_, como servicios de chat, registro de actividades y otros servicios necesarios para una aplicación o servicio inteligente.

In [ ]:
var builder = Kernel.CreateBuilder();
builder.AddAzureOpenAIChatCompletion(azureOpenAIDeployment, azureOpenAIEndpoint, azureOpenAIKey);

var kernel = builder.Build();

var chatService = kernel.GetRequiredService<IChatCompletionService>();

Primero, usemos el modelo directamente. Para simplemente llamar al modelo, podemos pasar una lista de mensajes al método `.GetChatMessageContentAsync`:

In [ ]:
Console.WriteLine(await chatService.GetChatMessageContentAsync("Hi! I'm Bob"));

El modelo por sí solo no tiene ningún concepto de estado. Por ejemplo, si haces una pregunta de seguimiento no sabrá de qué le estás hablando.

In [ ]:
Console.WriteLine(await chatService.GetChatMessageContentAsync("What's my name?"));

Para solucionar esto, necesitamos pasar todo el historial de la conversación al modelo. Veamos qué sucede cuando hacemos eso:

In [ ]:
var messages = new ChatHistory();
messages.AddUserMessage("Hi! I'm Bob");
messages.AddAssistantMessage("Hello Bob! How can I help you today?");
messages.AddUserMessage("What's my name?");

Console.WriteLine(await chatService.GetChatMessageContentAsync(messages));

¡Y ahora podemos ver que obtenemos una buena respuesta!

Esta es la idea básica que sustenta la capacidad de un chatbot para interactuar de manera conversacional. Entonces, ¿cómo implementamos esto de la mejor manera?

## _Prompt Engineering_

Un _prompt_ En este contexto, un prompt es un conjunto de instrucciones o información que se proporciona a un modelo de lenguaje (LLM) para guiar su respuesta. Los _prompts_ pueden incluir mensajes del usuario, mensajes del sistema con instrucciones específicas, o cualquier otra información relevante que ayude al modelo a generar una respuesta adecuada. Por ejemplo, en un chatbot, un _prompt_ podría incluir el historial de la conversación y un mensaje del sistema que le indique al modelo cómo debe comportarse o qué tono debe usar. Esto ayuda a que el modelo entienda mejor el contexto y genere respuestas más coherentes y útiles.

El escribir _prompts_ es tanto un arte como una ciencia, muy basada en el ensayo y el error, y totalmente dependiente de la naturaleza, tipo y versión del modelo de Inteligencia Artificial que estemos utilizando. Tanto es así que existe una disciplina al respecto llamada **_Prompt Engineering_**.

### Historial de la Conversación

La librería de Semantic Kernel implementa una capa de persistencia incorporada, lo que lo hace ideal para aplicaciones de chat que soportan múltiples turnos conversacionales. Este componente se llama `ChatHistory` y podemos informarlo con contenido formateado y almacenado en un repositorio de datos tal como un Azure SQL, Azure Cosmos DB o cualquier otro tipo de _backend_ de persistencia.

In [ ]:
var piratePrompt = "You talk like a pirate. Answer all questions to the best of your ability.";

var messages = new ChatHistory(piratePrompt);
messages.AddUserMessage("Hi! I'm Jim");

var response = await chatService.GetChatMessageContentAsync(messages);
messages.AddAssistantMessage(response.Content);

Console.WriteLine(response);

La clase `ChatHistory` sirve como un repositorio en memoria de la conversación hasta ahora con el modelo. No obstante, es responsabilidad nuestra mantenerla informada con con el resultado de la llamada al modelo.

In [ ]:
messages.AddUserMessage("What's my name?");

Console.WriteLine(await chatService.GetChatMessageContentAsync(messages));

## Plantillas de _prompts_

Las plantillas de _prompts_ son construcciones que ayudan a convertir la información cruda del usuario en un formato con el que el modelo de lenguaje (LLM) pueda trabajar. En este caso, la entrada cruda del usuario es solo un mensaje, que estamos pasando al LLM. Ahora hagamos esto un poco más complicado. Primero, agreguemos un mensaje del sistema con algunas instrucciones personalizadas (pero aún tomando mensajes como entrada). Luego, agregaremos más entradas además de los mensajes.

En este caso crearemos una plantilla que nos sirva para gestionar como se debe comportar el chatbot con funciones para la gestión del historial de la conversación.

In [ ]:
const string chatbotPrompt = @"
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$userInput}}
ChatBot:";

var executionSettings = new OpenAIPromptExecutionSettings 
{
    MaxTokens = 2000,
    Temperature = 0.7,
    TopP = 0.5
};

var chatbotFunction = kernel.CreateFunctionFromPrompt(chatbotPrompt, executionSettings);

var history = "";

var arguments = new KernelArguments()
{
    ["history"] = history
};

Func<string, Task> ChatAsync = async (string input) => {
    // Save new message in the arguments
    arguments["userInput"] = input;

    // Process the user message and get an answer
    var answer = await chatbotFunction.InvokeAsync(kernel, arguments);

    // Append the new interaction to the chat history
    var result = $"\nUser: {input}\nAI: {answer}\n";
    history += result;

    arguments["history"] = history;
    
    // Show the response
    Console.WriteLine(result);
};

Y procedemos a realizar varias interacciones:

In [ ]:
await ChatAsync("Hi, I'm looking for book suggestions");

In [ ]:
await ChatAsync("I would like a non-fiction book suggestion about Greece history. Please only list one book.");

In [ ]:
await ChatAsync("that sounds interesting, what are some of the topics I will learn about?");

In [ ]:
await ChatAsync("Which topic from the ones you listed do you think most people find interesting?");

In [ ]:
await ChatAsync("could you list some more books I could read about the topic(s) you mentioned?");

## _Streaming_

Ahora tenemos un chatbot funcional. Sin embargo, una consideración muy importante de la experiencia de usuario (UX, _User Experience_) para las aplicaciones de chatbot es el _streaming_. Los modelos de lenguaje (LLMs) a veces pueden tardar un tiempo en responder y, para mejorar la experiencia del usuario, una cosa que la mayoría de las aplicaciones hacen es transmitir cada token a medida que se genera. Esto permite al usuario ver el progreso tal como si la Inteligencia Artificial le estuviera escribiendo.

**¡En realidad es muy fácil hacer esto!**

Lo que haremos es emplear el método `InvokeStreamingAsync` para reaizar las llamadas, mostrando con cada recepción el token que se haya generado.

In [ ]:
Func<string, Task> ChatStreamingAsync = async (string input) => {
    // Save new message in the arguments
    arguments["userInput"] = input;

    var responseBuilder = new StringBuilder();

    // Process the user message and get an answer
    await foreach (var update in chatbotFunction.InvokeStreamingAsync(kernel, arguments))
    {
        Console.Write(update);   
        responseBuilder.Append(update);
    }

    // Append the new interaction to the chat history
    var result = $"\nUser: {input}\nAI: {responseBuilder}\n";
    history += result;

    arguments["history"] = history;
};


await ChatStreamingAsync("Now please suggest me some science fiction books");

In [ ]:
Console.WriteLine(history);